In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec  7 16:24:14 2018

@author: Xingguang Zhang
"""
import numpy as np
import pandas as pd

def create_dataset(dataset, numStepTrain, window_size, label_column):
    trainX, trainY, testX, testY = [], [], [], []
    numfeatures = np.size(dataset,1)
    if numStepTrain < 1:
        numStepTrain = int(len(dataset) * numStepTrain)
    #split out train set
    dataTrain = dataset[0:numStepTrain]
    #normalization
    sig, mu, dataTrainStd = normalize(dataTrain)
    #create train and test dataset
    for i in range(numStepTrain - window_size):
        a = dataTrainStd[i:(i+window_size), :]
        trainX.append(a)
        trainY.append(dataTrainStd[(i + 1) : (i + window_size + 1), label_column])
    for j in range((numStepTrain- window_size),(len(dataset)-window_size)):
        a = dataset[j : (j + window_size),:]
        testX.append(a)
        testY.append(dataset[(j + 1) : (j + window_size + 1), label_column])
    #reshape the dataset to the standard shape of LSTM input
    trainX, trainY, testX, testY = np.array(trainX), np.array(trainY), np.array(testX), np.array(testY)
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], numfeatures))
    testX = (testX - mu) / sig
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], numfeatures))
    testY = (testY - mu) / sig
    return sig, mu, trainX, trainY, testX, testY

def normalize(train_data):
    sig = np.std(train_data)
    mu = np.mean(train_data)
    dataTrainStd = (train_data - mu) / sig
    return sig, mu, dataTrainStd

def read_data(path, steps, no_column):
    whole = np.array(pd.read_csv(path))
    data = whole[::steps,no_column]
    data = np.reshape(data, [np.size(data,0),np.size(no_column)])
    for k in range(data.shape[1]):
        # for the unrecorded data(nan), padding it with that of the previous time step 
        for i in range(data.shape[0]):
            if np.isnan(data[i,k]):
                data[i,k] = data[i-1,k]
    return data.astype('float32')


In [4]:
pip install matplotlib

     |████████████████████████████████| 12.4 MB 6.5 MB/s eta 0:00:01
     |████████████████████████████████| 67 kB 5.1 MB/s  eta 0:00:01
     |████████████████████████████████| 88 kB 6.1 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install keras

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install data_creation

ERROR: Could not find a version that satisfies the requirement data_creation (from versions: none)
ERROR: No matching distribution found for data_creation
Note: you may need to restart the kernel to use updated packages.


In [10]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec  7 16:27:27 2018

@author: Xingguang Zhang
"""
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.metrics import mean_squared_error
from keras.models import Model
from keras.layers import Input
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from keras import optimizers
#from keras import regularizers
from data_creation import create_dataset, read_data
from dual_attention import first_attention, second_attention


ModuleNotFoundError: No module named 'data_creation'

In [2]:

def scheduler(epoch):
    # every 20 epochs, lr reduced to 20%
    if epoch % 20 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.2)
        print("lr changed to {}".format(lr * 0.2))
    return K.get_value(model.optimizer.lr)

np.random.seed(8)
path = 'full_non_padding.csv'
#sample the data for every 20 minutes to reduce the amount of data
sample_step = 5
#apple, adobe, amazon,microsoft,netflix
in_column = (1,2,11,62,67)
out_column = 67

for i in range(np.size(in_column)):
    in_column = list(in_column)
    label_column = 0
    if in_column[i] == out_column:
        label_column = i
        break
        
data = read_data(path, sample_step, in_column)
data_attention = np.reshape(data[:,label_column] ,(-1, 1))
#create dataset
numFeature = np.size(data,1) #n
#Time steps (window size)
T = 20
#cells number in encoder
m = 64
#cells number in decoder
p = 64
# prepare inputs
sig, mu, trainX, trainY, testX, testY = create_dataset(data,\
                        numStepTrain = 0.7, window_size = T, label_column = label_column)

train_label = np.reshape(trainY[:, -1], (-1, 1))
test_label = np.reshape(testY[:, -1], (-1, 1))
h_init = np.zeros((trainX.shape[0], m))
s_init = np.zeros((trainX.shape[0], m))
hd_init = np.zeros((trainX.shape[0], p))
sd_init = np.zeros((trainX.shape[0], p))

# define input variables
In1 = Input(shape = (trainX.shape[1], trainX.shape[2]))         #(None, T, m)
Iny = Input(shape = (trainY.shape[1],))                         #(None, T)
h0 = Input(shape = (m,))                                        #(None, m)
s0 = Input(shape = (m,))
hd0 = Input(shape = (p,))                                       #(None, m)
sd0 = Input(shape = (p,))

#attention 1st stage --- encoder
H = first_attention(In1, h0, s0, m, T, numFeature)              #(None, T, m)
#attention 2nd stage
Y = second_attention(H, Iny, hd0, sd0, m, p, T)              

reduce_lr = LearningRateScheduler(scheduler)
#build model
model = Model(inputs = [In1, Iny, h0, s0, hd0, sd0], outputs = Y)
optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='mean_squared_error', optimizer = optimizer)
#model.summary()
model.fit([trainX, trainY, h_init, s_init, hd_init, sd_init], train_label, epochs=120, batch_size=128, \
          callbacks=[reduce_lr], verbose=1, shuffle=False)

trainPredict = model.predict([trainX, trainY, h_init, s_init, hd_init, sd_init])
testPredict = model.predict([testX, testY, h_init, s_init, hd_init, sd_init])

trainPredict = trainPredict * sig + mu
train_label = train_label * sig + mu
testPredict = testPredict * sig + mu
test_label = test_label * sig + mu

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(train_label, trainPredict))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(test_label, testPredict))
print('Test Score: %.2f RMSE' % (testScore))

trainPredictPlot = np.empty_like(data_attention)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[T : len(trainPredict) + T, :] = trainPredict
# shift test predictions for plotting
testPredictPlot = np.empty_like(data_attention)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict) + T : len(data_attention), :] = testPredict
# plot baseline and predictions
plt.figure(1,figsize=(12,9))
plt.plot(data_attention, label = 'ground truth')
plt.plot(trainPredictPlot, label = 'train prediction')
plt.plot(testPredictPlot, label = 'test prediction')
plt.legend()
plt.show()
plt.figure(1, figsize=(12, 9))
plt.plot(test_label, label = 'test target')
plt.plot(testPredict, label = 'prediction')
plt.legend()
plt.show()


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 10 14:37:25 2018

@author: Xingguang Zhang
"""
from keras.layers import LSTM,Dense,Activation,Softmax,Lambda,RepeatVector,\
            Add,Reshape,Multiply,Concatenate,Dot
import keras.backend as K
from keras.constraints import maxnorm
from keras import regularizers

def first_attention(In1, h0, s0, m, T, numFeature):
    X_inv = Lambda(lambda x: K.permute_dimensions(In1, (0,2,1)))(In1)  
    UedotXk = Dense(T, use_bias = False)(X_inv)
    for t in range(T):
        x = Lambda(lambda x: In1[:,t,:])(In1)  
        if t == 0:
            x_lstm = Reshape((1,numFeature))(x)
            h, _, s = LSTM(m, return_state = True)(x_lstm, initial_state = [h0, s0])
            H = Reshape((1, m))(h)
        else:
            x_lstm = Reshape((1,numFeature))(x_lstm)
            h, _, s = LSTM(m, kernel_constraint=maxnorm(4.),kernel_regularizer = regularizers.l2(0.01),\
                           return_state = True)(x_lstm, initial_state = [h, s])
            ht = Reshape((1, m))(h)
            H = Concatenate(axis = 1)([H, ht])
        conc = Concatenate()([h, s])
        WedotHs = Dense(T, use_bias = False)(conc)
        WedotHs = RepeatVector(numFeature)(WedotHs)
        add = Add()([WedotHs, UedotXk])
        acti = Activation(activation = 'tanh')(add)
        e = Dense(1, use_bias = False)(acti)
        e = Reshape((numFeature,))(e)
        alpha = Softmax(axis = 1)(e)
        x_lstm = Multiply()([x, alpha])
    return H

def second_attention(H, Iny, hd0, sd0, m, p, T):
    UddotXk = Dense(m, use_bias = False)(H)
    d = hd0
    c = sd0
    for t in range(T-1):
        y = Lambda(lambda x: Iny[:,t])(Iny) 
        y = Reshape((1,))(y)
        conc = Concatenate()([d, c]) 
        WddotHs = Dense(m, use_bias = False)(conc)
        WddotHs = RepeatVector(T)(WddotHs)
        add = Add()([WddotHs, UddotXk])
        acti = Activation(activation = 'tanh')(add)
        l = Dense(1, use_bias = False)(acti)
        l = Reshape((T,))(l)
        beta = Softmax(axis = 1)(l)
        context = Dot(axes = 1)([beta,H])                                       #(None,m)
        cy = Concatenate()([y, context])
        new_y = Dense(1, use_bias = True)(cy)
        new_y = Reshape((1, 1))(new_y)
        d, _, c = LSTM(p, kernel_constraint=maxnorm(4.),kernel_regularizer = regularizers.l2(0.01),\
                       return_state = True)(new_y, initial_state = [d, c])
    dc = Concatenate()([d, c]) 
    Wydotdc = Dense(p, use_bias = True)(dc)
    Y = Dense(1, use_bias = True)(Wydotdc)
    return Y